<a href="https://colab.research.google.com/github/dkswndms4782/Deep-Learning/blob/main/HAR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/gdrive')
drive.mount('/content/gdrive')

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
#tensor data여야함

import numpy as np 
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
Mounted at /content/gdrive


In [ ]:
class TrainDataset(Dataset): 
  # Initialize your data, download, etc. 
  def __init__(self): 
    xy = pd.read_csv('./gdrive/My Drive/HAR CNN/train.csv')
    i = 0
    for row in xy['Activity']:
      if row == "WALKING":
        xy['Activity'][i] = 0
        i += 1
      elif row == "WALKING_UPSTAIRS":
         xy['Activity'][i] = 1
         i += 1
      elif row == "WALKING_DOWNSTAIRS":
          xy['Activity'][i] = 2
          i += 1
      elif row == "SITTING":
          xy['Activity'][i] = 3
          i += 1
      elif row == "STANDING":
          xy['Activity'][i] = 4
          i += 1
      elif row == "LAYING":
          xy['Activity'][i] = 5
          i += 1
      else :
        i += 1
    xy = np.array(xy,dtype = np.double())
    xy = np.expand_dims(xy,axis = 2)
    self.len = xy.shape[0] 
    self.x_data = torch.from_numpy(xy[:, 0:-1]) 
    self.y_data = torch.from_numpy(xy[:, [-1]]) 
    
  def __getitem__(self, index): 
    return self.x_data[index], self.y_data[index] 
    
  def __len__(self): 
    return self.len


In [ ]:
class TestDataset(Dataset): 
  # Initialize your data, download, etc. 
  def __init__(self): 
    xy = pd.read_csv('./gdrive/My Drive/HAR CNN/test.csv')
    i = 0
    for row in xy['Activity']:
      if row == "WALKING":
        xy['Activity'][i] = 0
        i += 1
      elif row == "WALKING_UPSTAIRS":
         xy['Activity'][i] = 1
         i += 1
      elif row == "WALKING_DOWNSTAIRS":
          xy['Activity'][i] = 2
          i += 1
      elif row == "SITTING":
          xy['Activity'][i] = 3
          i += 1
      elif row == "STANDING":
          xy['Activity'][i] = 4
          i += 1
      elif row == "LAYING":
          xy['Activity'][i] = 5
          i += 1
      else :
        i += 1
    xy = np.array(xy,dtype = np.double())
    xy = np.expand_dims(xy,axis = 2)
    self.len = xy.shape[0] 
    self.x_data = torch.from_numpy(xy[:, 0:-1]) 
    self.y_data = torch.from_numpy(xy[:, [-1]]) 
    
  def __getitem__(self, index): 
    return self.x_data[index], self.y_data[index] 
    
  def __len__(self): 
    return self.len


In [ ]:
train = TrainDataset()
test = TestDataset()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be

In [ ]:
#parameters
learning_rate = 0.01
batch_size = 100
epochs = 1000

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.keep_prob = 0.5
    self.layer1 = nn.Sequential(
        nn.Conv1d(562,32,kernel_size = 3,stride =1, padding = 1),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size = 2,stride = 2,padding=1)
    )
    self.layer2 = nn.Sequential(
        nn.Conv1d(32,64,kernel_size = 3,stride = 1,padding = 1),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size = 2,stride = 2,padding=1)
    )
    self.layer3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding=1))
    self.fc1 = nn.Linear(128,128,bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(p = 1 - self.keep_prob))
    self.fc2 = nn.Linear(128,6,bias = True)
    nn.init.xavier_uniform_(self.fc2.weight)
    

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0),-1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [ ]:
model = CNN().to(device)
model = model.double()

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
total_batch = train.len

In [ ]:
"""
for epoch in range(epochs):
  avg_cost = 0
  for X,Y in train:
    X = X.to(device)
    Y = Y.to(device)
    Y = np.squeeze(Y,axis = 1)
    Y = torch.tensor(Y)
    Y = Y.long()

    X = np.expand_dims(X,axis = 0)
    X = torch.tensor(X)
    optimizer.zero_grad()
    hypothesis = model(X)


    cost = criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()
    
    avg_cost += cost / total_batch
  print('[Epoch : {}] cost = {}'.format(epoch+1,avg_cost))
print('Learning Finished!')
"""

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


[Epoch : 1] cost = 2.588251278335145
[Epoch : 2] cost = 2.2576616348885574
[Epoch : 3] cost = 1.927013764485113
[Epoch : 4] cost = 1.8883406158664289
[Epoch : 5] cost = 1.9227959440173283
[Epoch : 6] cost = 1.9079118008803153
[Epoch : 7] cost = 2.0490296582287866
[Epoch : 8] cost = 1.9004804367138932
[Epoch : 9] cost = 1.9330782404667983
[Epoch : 10] cost = 1.895991482825986
Learning Finished!


In [ ]:

for epoch in range(epochs):
  avg_cost = 0
  X = train.x_data.to(device)
  Y = train.y_data.to(device)
  Y = np.squeeze(Y,axis = 1)
  Y = np.squeeze(Y,axis = 1)
  Y = torch.tensor(Y)
  Y = Y.long()


  optimizer.zero_grad()
  hypothesis = model(X)
  hypothesis = np.squeeze(hypothesis,axis = 1)


  cost = criterion(hypothesis,Y)
  cost.backward()
  optimizer.step()
    
  avg_cost += cost / total_batch
  print('[Epoch : {}] cost = {}'.format(epoch+1,avg_cost))
print('Learning Finished!')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


[Epoch : 1] cost = 0.0002446363459010025
[Epoch : 2] cost = 0.0002444471464939795
[Epoch : 3] cost = 0.0002378157988853305
[Epoch : 4] cost = 0.000220368216504378
[Epoch : 5] cost = 0.00020023810049581375
[Epoch : 6] cost = 0.00018584987963633212
[Epoch : 7] cost = 0.00017524707456374023
[Epoch : 8] cost = 0.00016671197139673616
[Epoch : 9] cost = 0.00016319057222627492
[Epoch : 10] cost = 0.00015940238248182806
[Epoch : 11] cost = 0.00015291549389262284
[Epoch : 12] cost = 0.00014895965586451628
[Epoch : 13] cost = 0.00014420196835218427
[Epoch : 14] cost = 0.00013842433486408736
[Epoch : 15] cost = 0.0001412417884686422
[Epoch : 16] cost = 0.0001263331234369032
[Epoch : 17] cost = 0.00011514092864781252
[Epoch : 18] cost = 0.00011240916040827018
[Epoch : 19] cost = 0.00011302044473696832
[Epoch : 20] cost = 0.0001137301019747027
[Epoch : 21] cost = 9.751779438969813e-05
[Epoch : 22] cost = 0.00010034849145863684
[Epoch : 23] cost = 9.215628744811803e-05
[Epoch : 24] cost = 9.58525751

KeyboardInterrupt: ignored

In [ ]:
with torch.no_grad():
  X_test = test.x_data.double().to(device)
  Y_test = test.y_data.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction,1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:',accuracy.item())

Accuracy: 0.16741186380386353
